In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as image
import glob
import os

In [ ]:
images_fp = "./images"

In [ ]:
images_names = [os.path.basename(file) for file in glob.glob(os.path.join(images_fp, '*.jpg'))]

In [ ]:
labels = [' '.join(name.split('_')[:-1:]) for name in images_names]

In [ ]:
labels

In [ ]:
def label_encode(labels):
    if label == 'Abyssinian': return 0
    if label == 'american pit bull terrier': return 1
    if label == 'american bulldog': return 2
    if label == 'basset hound': return 3
    if label == 'beagle': return 4
    if label == 'Bengal': return 5
    if label == 'Bombay': return 6
    if label == 'British Shorthair': return 7
    if label == 'Egyptian Mau': return 8
    if label == 'boxer': return 9
    if label == 'chihuahua': return 10
    if label == 'english cocker spaniel': return 11
    if label == 'english setter': return 12
    if label == 'german shorthaired': return 13
    if label == 'great pyreness': return 14
    if label == 'Birman': return 15

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
features = []
labels = []
IMAGE_SIZE = (224,224)

In [ ]:
for name in images_names:
    label = ' '.join(name.split('_')[:-1:])
    label_encoded = label_encode(label)
    if label_encoded != None :
        img = load_img(os.path.join(images_fp, name))
        img = tf.image.resize_with_pad(img_to_array(img, dtype='uint8'),*IMAGE_SIZE).numpy().astype('uint8')
        image = np.array(img)
        features.append(image)
        labels.append(label_encoded)
        
    

In [ ]:
features_array = np.array(features)
labels_array = np.array(labels)

In [ ]:
labels_one_hot = pd.get_dummies(labels_array,dtype = int)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features_array,labels_one_hot,test_size = 0.2,random_state = 42)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.25,random_state = 1)

In [ ]:
from tensorflow.keras import layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as pp_i
from tensorflow.keras.layers import RandomFlip,RandomRotation,Dense,Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
data_augmentation = Sequential([RandomFlip("horizontal_and_vertical"),RandomRotation(0.2)])
prediction_layers = Dense(15,activation = 'softmax')

In [ ]:
resnet_model = ResNet50(include_top = False,pooling = 'avg' , weights = 'imagenet')
resnet_model.trainable = False
preprocess_input = pp_i

In [ ]:
inputs = Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = resnet_model(x, training = False)
x = Dropout(0.2)(x)
outputs = prediction_layers(x)
model = Model(inputs,outputs)

In [ ]:
model.compile(optimizer=Adam(), loss = CategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(x=X_train, y=y_train , validation_data =(X_val, y_val), epochs =10)

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

In [ ]:
epochs_range = range(10)
plt.figure(figsize=(15,8)) 

plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training accuracy')
plt.plot(epochs_range,val_acc,label='Validation accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')


In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred